In [ ]:
import pandas as pd
import random as rn
import numpy as np
import os
#Change directory to parent directory if necessary
if os.getcwd() == '/home/USACE_Modeling':
    None
else:
    os.chdir(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
    
import sys
import ast

from pathlib import Path
import matplotlib.pyplot as plt
SEED = 1234
np.random.seed(SEED)
rn.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" #Use CPU

import tensorflow as tf
rn.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras import backend as K
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.losses import Huber, MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import Adam
import optuna
import optuna.storages
from optuna.storages import RetryFailedTrialCallback
from optuna.study import MaxTrialsCallback
from optuna.trial import TrialState
par_dir = os.getcwd() #Parent Directory
par_dir = Path(par_dir)
sys.path.append(str(par_dir))
from Functions import create_dataset

## Forecasting

In [ ]:
prediction_folder = 'Result_3_1'
#Read Main Data
wcs_df = pd.read_csv(par_dir / 'Data' / 'UARK_WCS_AIS_Compiled_NewData.csv')
cons_loc = pd.read_csv(par_dir / 'Data' / 'location_with_consistent_data_newdata.csv')
wcs_df = pd.merge(wcs_df,cons_loc,how='left',on='sub_folder')

terminal_type = wcs_df[['sub_folder','dominant_vessel_type']].drop_duplicates()
comm_code = pd.read_csv(par_dir / 'Data' / 'WCSC_Commodity_CodesList.csv')
#Keep unique SingleDigitCode and SingleDigitDescription from comm_code
comm_code = comm_code[['SingleDigitCode','SingleDigitDescription']].drop_duplicates()

prediction_combined_list = []
prediction_combined_df = pd.DataFrame()
#List of data files
data_model_dir = {1:'UARK_WCS_AIS_Compiled_NewData_No_Aggregation.csv', 2:'UARK_WCS_AIS_Compiled_NewData_Mixed.csv', 3:'UARK_WCS_AIS_Compiled_NewData_Self-Propelled, Dry Cargo.csv',4:'UARK_WCS_AIS_Compiled_NewData_Self-Propelled, Tanker.csv',5:'UARK_WCS_AIS_Compiled_NewData_Tug_Tow.csv'}

data_model_dir = {1:'UARK_WCS_AIS_Compiled_NewData_No_Aggregation.csv'}

for i_filenumber in data_model_dir.keys():
    #Read data
    if os.getcwd() == '/home/USACE_Modeling':
        data_num = int(sys.argv[1]) #For HPC
    else:
        data_num=i_filenumber
    #Read Data
    file_loc = par_dir / 'Data' / data_model_dir[data_num]
    df = pd.read_csv(file_loc)
    #Drop columns that start with 'dwell
    df = df.drop(columns=[col for col in df.columns if col.lower().startswith('dwell_'.lower())])
    
    #Removing outliers
    outlier_terminal = pd.read_csv(par_dir / 'Data' / 'outlier_terminals.csv')
    #Remove records from outlier terminals whose sub_folder are in outlier_terminal['terminal']
    df = df[~df['sub_folder'].isin(outlier_terminal['terminal'])]
    
    #Data processing
    df['sin_quarter'] = np.sin(2*np.pi*(df['QuarterOfTheYear']-0)/4)
    df['cos_quarter'] = np.cos(2*np.pi*(df['QuarterOfTheYear']-0)/4)
    df = df.drop(columns=['QuarterOfTheYear'])
    target = list(df.columns.values)
    target = [idx for idx in target if idx.lower().startswith('C_'.lower())]
    #Parameters
    end_training = '2019Q1'
    begin_validation = '2018Q1'
    end_validation = '2020Q1'
    begin_testing = '2019Q1'
    end_testing = '2020Q4'
    n_past = 4 #Look back period
    n_future = 4 #Number of period to forecast each time
    batch_size = 1 #For creating dataset
    n_epochs = 1000
    df = df.set_index(['quarter'])
    df = df.loc[df.index <= end_testing]
    #Reorder columns
    df = df[target + [col for col in df.columns if col not in target]]
    n_unknown_features = len(target)
    n_features = len([x for x in list(df.columns.values) if x not in ['quarter','sub_folder']])
    n_deterministic_features = n_features-n_unknown_features
    train_columns_list = [x for x in list(df.columns.values) if x not in ['quarter','sub_folder']]
        
    ## Split and Scale
    location_list = df[['sub_folder']].drop_duplicates()['sub_folder'].to_list()
    train_df = []
    val_df = []
    test_df = []
    for location in location_list:
        temp_df = df[df['sub_folder']==location].copy()
        temp_df = temp_df[train_columns_list]
        temp_train = temp_df.loc[:end_training].iloc[:-1]
        temp_val = temp_df.loc[begin_validation:end_validation].iloc[:-1]
        temp_test = temp_df.loc[begin_testing:]
        train_df.append(temp_train)
        val_df.append(temp_val)
        test_df.append(temp_test)
    train_df = pd.concat(train_df)
    test_df = pd.concat(test_df)
    val_df = pd.concat(val_df)
    true_labels = df[target+['sub_folder']][df[target+['sub_folder']].index.isin(test_df.index.unique()[n_past:].to_list())].copy()
    true_lables_train = df[target+['sub_folder']][df[target+['sub_folder']].index.isin(train_df.index.unique()[n_past:].to_list())].copy()
    true_labels_val = df[target+['sub_folder']][df[target+['sub_folder']].index.isin(val_df.index.unique()[n_past:].to_list())].copy()
    # Scale the data
    #Scale features
    features = [x for x in list(train_df.columns.values) if x not in target]
    #List of columns that start with stop or dwell
    ais_features = [col for col in features if col.lower().startswith('stop'.lower()) or col.lower().startswith('dwell'.lower())]
    feature_scaler = MinMaxScaler()
    other_features = [x for x in features if x not in ais_features]
    #Scale other features
    train_df[other_features] = feature_scaler.fit_transform(train_df[other_features])
    val_df[other_features] = feature_scaler.transform(val_df[other_features])
    test_df[other_features] = feature_scaler.transform(test_df[other_features])
    #Scale AIS features
    ais_min = train_df[ais_features].min().min()
    ais_max = train_df[ais_features].max().max()
    train_df[ais_features] = (train_df[ais_features] - ais_min)/(ais_max - ais_min)
    val_df[ais_features] = (val_df[ais_features] - ais_min)/(ais_max - ais_min)
    test_df[ais_features] = (test_df[ais_features] - ais_min)/(ais_max - ais_min)
    #Scale target
    train_min = train_df[target].min().min()
    train_max = train_df[target].max().max()
    train_df[target] = (train_df[target] - train_min)/(train_max - train_min)
    val_df[target] = (val_df[target] - train_min)/(train_max - train_min)
    test_df[target] = (test_df[target] - train_min)/(train_max - train_min)
    #Create dataset
    val_windowed = create_dataset(val_df, n_deterministic_features, n_past, n_future,batch_size, target)
    training_windowed = create_dataset(train_df, n_deterministic_features, n_past, n_future,batch_size,target)
    test_windowed = create_dataset(test_df, n_deterministic_features, n_past, n_future,batch_size, target)
    
    study_name = data_model_dir[data_num].split('.')[0]
    #Load and compile model
    #Find file name that starts with study_name and ends with .h5
    model_path =par_dir / 'Outputs' / 'LSTM_Outputs' / prediction_folder
    for filename in os.listdir(model_path):
        if filename.startswith(study_name) and filename.endswith('.h5'):
            model_name = filename
    print(model_name)
    
    storage_path = "Outputs/LSTM_Outputs/" + prediction_folder + "/LSTM_study.log"
    storage_url = optuna.storages.JournalStorage(optuna.storages.JournalFileStorage(storage_path),)
    study = optuna.study.load_study(study_name = study_name, storage=storage_url)
    study_temp_df = study.trials_dataframe()
    #Get best trial
    best_trial = study_temp_df[study_temp_df['value']==study_temp_df['value'].min()]
    
    
    
    model = load_model(model_path / model_name, compile=False)
    model.compile(loss=Huber(), optimizer=best_trial['params_optimizer'].to_list()[0], metrics='mse')
    
    temp_pred_list = []
    temp_pred_df = pd.DataFrame()
    
    actuals, forecasts = [], []
    for i, ((past, future), actual) in enumerate(test_windowed):
        pred = model.predict((past, future), verbose=0, batch_size=1500)
        actuals.append(actual.numpy().flatten())
        forecasts.append(pred.flatten())
    print('Predict completed')
    #convert actuals to dataframe
    actuals_df = pd.DataFrame(actuals)
    #Convert to only one column starting from first row
    actuals_df = actuals_df.stack().reset_index(drop=True)
    actuals_df = pd.DataFrame(actuals_df, columns=['Actuals'])
    
    # #Convert forecasts to dataframe
    forecasts_df = pd.DataFrame(forecasts)
    #Convert to only one column starting from first row
    forecasts_df = forecasts_df.stack().reset_index(drop=True)
    forecasts_df = pd.DataFrame(forecasts_df, columns=['Predictions'])
    
    #Concatenate actuals and forecasts
    actuals_predictions_df = pd.concat([actuals_df, forecasts_df], axis=1)
    actuals_predictions_df
    
    #Rescale actuals and predictions using train_min and train_max
    actuals_predictions_df['Actuals'] = (actuals_predictions_df['Actuals']*(train_max - train_min) + train_min)
    actuals_predictions_df['Predictions'] = (actuals_predictions_df['Predictions']*(train_max - train_min) + train_min)
    
    #Set actuals to int
    actuals_predictions_df['Actuals'] = actuals_predictions_df['Actuals'].round(0).astype(int)
    #Create dataframe
    temp_label_df = []
    true_labels.reset_index(inplace=True)
    #Iterate through each row of true_labels
    for index, row in true_labels.iterrows():
        quarter = row['quarter']
        commodity = row[target]
        sub_folder = row['sub_folder']
        #Create dataframe using quarter, commodity, sub_folder
        temp_df = pd.DataFrame({'quarter':quarter,'Commodity':target, 'Actuals':commodity,'sub_folder':sub_folder})
        #Append to temp_label_df
        temp_label_df.append(temp_df)
    #Concatenate temp_label_df
    temp_label_df = pd.concat(temp_label_df)
    #Set actuals to int
    temp_label_df['Actuals'] = temp_label_df['Actuals'].astype(int)
    temp_label_df = temp_label_df.reset_index().drop(columns=['index'])
    
    #Merge actuals_predictions_df with temp_label_df on index
    final_predictions_df = pd.merge(temp_label_df,actuals_predictions_df,how='left',left_index=True,right_index=True)
    
    #Raise error if Actuals_x and Actuals_y are not equal
    if (final_predictions_df['Actuals_x'] != final_predictions_df['Actuals_y']).any():
        raise ValueError('Actuals_x and Actuals_y are not equal')
    #Drop Actuals_y
    final_predictions_df = final_predictions_df.drop(columns=['Actuals_y'])
    #Rename Actuals_x to Actuals
    final_predictions_df = final_predictions_df.rename(columns={'Actuals_x':'Actuals'})
    final_predictions_df['Model'] = 'LSTM'
    final_predictions_df['Aggregation'] = study_name
    prediction_combined_list.append(final_predictions_df)
    
prediction_combined_df = pd.concat(prediction_combined_list)
#Removing outliers
outlier_terminals_commodity = pd.read_csv(par_dir / 'Data' / 'outlier_terminals_commodity.csv')
#Rename terminal to sub_folder and commodity to Commodity
outlier_terminals_commodity = outlier_terminals_commodity.rename(columns={'terminal':'sub_folder','commodity':'Commodity'})
#Remove records from prediction_combined_df whose sub_folder and commodity are in outlier_terminals_commodity in the same row
prediction_combined_df = prediction_combined_df[~prediction_combined_df[['sub_folder','Commodity']].apply(tuple,1).isin(outlier_terminals_commodity[['sub_folder','Commodity']].apply(tuple,1))]
#Keep string after last _ in Aggregation
prediction_combined_df['Aggregation'] = prediction_combined_df['Aggregation'].str.split('NewData_').str[-1]

In [ ]:
#Keep row with first lowest value from best_trial
best_hp = best_trial[best_trial['value']==best_trial['value'].min()].iloc[0]
#Print best hyperparameters
print('Best hyperparameters:')
for key, value in best_hp.to_dict().items():
    print('    {}: {}'.format(key, value))


In [ ]:
prediction_combined_df[(prediction_combined_df['Aggregation']=='No_Aggregation') & (prediction_combined_df['Predictions']<0)]

In [ ]:
prediction_combined_df[(prediction_combined_df['Aggregation']=='No_Aggregation') & (prediction_combined_df['sub_folder']=='PortElizabethBerthsNo52_98') & (prediction_combined_df['Commodity']=='C_2')]

In [ ]:
### prediction_combined_df.to_csv(par_dir / 'Outputs' / 'LSTM_Outputs' / prediction_folder /  'LSTM_Predictions.csv', mode='x', index=False)

In [ ]:
import shap

In [ ]:
type(training_windowed)

In [ ]:
explainer = shap.Explainer(model, train)